In [ ]:
#!/usr/bin/env python3

from Crypto.Util.number import isPrime, getPrime, long_to_bytes, bytes_to_long
from Crypto.Random.random import getrandbits, randint
from Crypto.Hash import SHA512

LOOP_LIMIT = 2000


def hash(val, bits=1024):
    output = 0
    for i in range((bits//512) + 1):
        h = SHA512.new()
        h.update(long_to_bytes(val) + long_to_bytes(i))
        output = int(h.hexdigest(), 16) << (512 * i) ^ output
    return output

In [37]:
def gen_snore_group(N=512):
    q = getPrime(N)
    for _ in range(LOOP_LIMIT):
        X = getrandbits(2*N)
        p = X - X % (2 * q) + 1
        if isPrime(p):
            break
    else:
        raise Exception("Failed to generate group")

    r = (p - 1) // q 

    for _ in range(LOOP_LIMIT):
        h = randint(2, p - 1)
        if pow(h, r, p) != 1:
            break
    else:
        raise Exception("Failed to generate group")

    g = pow(h, r, p)

    return (p, q, g)


def snore_gen(p, q, g, N=512):
    x = randint(1, q - 1)
    y = pow(g, -x, p)
    return (x, y)


def snore_sign(p, q, g, x, m):
    k = randint(1, q - 1)
    r = pow(g, k, p)
    e = hash((r + m) % p) % q
    s = (k + x * e) % q
    return (s, e)


def snore_verify(p, q, g, y, m, s, e):
    if not (0 < s < q):
        return False

    rv = (pow(g, s, p) * pow(y, e, p)) % p
    ev = hash((rv + m) % p) % q

    return ev == e


def main():
    p, q, g = gen_snore_group()
    print(f"p = {p}")
    print(f"q = {q}")
    print(f"g = {g}")

    queries = []
    for _ in range(10):
        x, y = snore_gen(p, q, g)
        print(f"y = {y}")

        print('you get one query to the oracle')

        m = int(input("m = "))
        queries.append(m)
        s, e = snore_sign(p, q, g, x, m)
        print(f"s = {s}")
        print(f"e = {e}")

        print('can you forge a signature?')
        m = int(input("m = "))
        s = int(input("s = "))
        # you can't change e >:)
        if m in queries:
            print('nope')
            return
        if not snore_verify(p, q, g, y, m, s, e):
            print('invalid signature!')
            return
        queries.append(m)
        print('correct signature!')

    print('you win !')
    print(open('flag.txt').read())

In [149]:
cmd = "ncat --ssl snore-signatures.chal.uiuc.tf 1337"
r = process(cmd,shell=True,stdin=PTY)

[x] Starting local process '/bin/sh'
[+] Starting local process '/bin/sh': pid 9370


In [150]:
r1=r.recvuntil("m =")
p = int(r1.decode().split("\n")[0][3:])
r.sendline("0")
r2=r.recvuntil("m =")
s=int(r2.decode().split("\n")[0][4:])
r.sendline(str(p*1))
r.recvuntil("s =")
r.sendline(str(s))

/tmp/ipykernel_6962/2776124356.py:1: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r1=r.recvuntil("m =")


b'p = 146916528656808197493772647519181531193620170526459935316687549229840589151444773617777412259002172547204128925700491466346168566944287420588360106256336672309981752990341903232284399727789948445654143038504888018974782792387483866105458140393120413168919675916009018114029215164068508729163576850522075369371\nq = 9665625040246828734882231466046617685246763193689137614264376726980574158262131140687099671481317629621448362372009776322763689236134418574462644425306927\ng = 107389919790093926901109707005694035177630165601173171897252694796663133510683873768364516719759396353977959652730980805181233021435205857310368897602322561946177565352234480231629411892649283797126404188397046746016909013489040934223523736258628543822315856728116651168744103810899807102807811325100030273734\ny = 139476670763060290412532877609508658925557557328531785206252985217492280246590513872931494892241724956301737534597228304383937535061581073544869925808542305491235579714357087715045423427080223506084645759

/tmp/ipykernel_6962/2776124356.py:4: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline("0")
/tmp/ipykernel_6962/2776124356.py:5: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r2=r.recvuntil("m =")


b' s = 883496782932158770111588904757009491479486711158763852977603514975391211163216788826984814045738023478823626001273253606133009635223544458152128432368261\ne = 4683289304844345055721609356089707983804632849454303757207924568254031539959611835228714893693730208077746941775003982146816149414852031878542633869401388\ncan you forge a signature?\nm ='


/tmp/ipykernel_6962/2776124356.py:8: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(str(p*1))
/tmp/ipykernel_6962/2776124356.py:9: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.recvuntil("s =")
/tmp/ipykernel_6962/2776124356.py:10: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(str(s))


In [151]:
for i in range(2,11):
    r1=r.recvuntil("m =")
    r.sendline("0")
    r2=r.recvuntil("m =")
    print(r2)
    s=int(r2.decode().split("\n")[0][4:])
    r.sendline(str(p*i))
    r.recvuntil("s =")
    r.sendline(str(s))

/tmp/ipykernel_6962/656721268.py:2: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r1=r.recvuntil("m =")
/tmp/ipykernel_6962/656721268.py:3: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline("0")
/tmp/ipykernel_6962/656721268.py:4: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r2=r.recvuntil("m =")


b' s = 2847714000286459314097245380477022781113506579943208776557079611122493744872240135794590714289526397012016899652396546889624466535439113006193249284948060\ne = 2124809275469295196418532735914273484584287188979326210153079571367072037668523366003255992776861555260143169476775833655816200332713698076876725965690740\ncan you forge a signature?\nm ='


/tmp/ipykernel_6962/656721268.py:7: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(str(p*i))
/tmp/ipykernel_6962/656721268.py:8: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.recvuntil("s =")
/tmp/ipykernel_6962/656721268.py:9: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendline(str(s))


b' s = 2457454682465008534252359892641550737065610222322882412802744466921026906724245912114660933855535548865173163930635327436854631799151788639413919511272875\ne = 6269085674307393493866611950208916943538210244023825300272737876189854710132123673671899795337537346587048127437874996975906939841153105981418400511251459\ncan you forge a signature?\nm ='
b' s = 812564315757629933979971765726826566468407186754929993408028662054583742421322922726470793046441153148545391188970060794200247316689088984624907721978776\ne = 3105360417446449979485828540708952607619927442779104293354627736606248061511796271738906384202135681906390825076812441362513158179787474150341343426547082\ncan you forge a signature?\nm ='
b' s = 4953577152038192086073356868778904830472630874525538923489530305321465838977177744204975574948469735045696730202920234006600891736886506650955198360140157\ne = 77395405343400852323185567789872000327160591215285939029543122415109584979172232448202145772575594825985322849531345625045

In [153]:
r.recv()

b'correct signature!\nyou win!\nuiuctf{add1ti0n_i5_n0t_c0nc4t3n4ti0n}\n\n'